<a href="https://colab.research.google.com/github/ahmedmabrouk24/Nueral-Network/blob/main/Neural_Network_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def import_data():
    from zipfile import ZipFile

    with ZipFile('gdrive/MyDrive/NN Dataset.zip') as zipObj:

      zipObj.extractall('gdrive/MyDrive/data')
#import_data()

In [2]:
pip install tflearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install imgaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install ipyplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers,models,Sequential
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.saving.save import load_model
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf
import cv2
import random
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import ipyplot
from PIL import Image
import shutil
import json


Instructions for updating:
non-resource variables are not supported in the long term



        WARNING! Google Colab Environment detected!
        You might encounter issues while running in Google Colab environment.
        If images are not displaying properly please try setting `force_b64` param to `True`.
        


In [7]:

TRAIN_DIR = 'drive/MyDrive/data/Train'
TEST_DIR = 'drive/MyDrive/data/Test'
train_images=[]
test_images=[]
val_images=[]
IMG_SIZE = 50
num_classes = 6
MODEL_NAME = 'sports_classification-cnn'


In [8]:
def convert_all_image_to_jpg():
    for img in tqdm(os.listdir(TRAIN_DIR)):
        x = img.split(".")
        path = 'drive/MyDrive/data/Train/' + img
        name = 'drive/MyDrive/data/Train/' + x[0]
        im = Image.open(path)
        im.convert('RGB').save(name + ".jpg")
        if (x[1] != 'jpg'):
             os.remove(path) 
#convert_all_image_to_jpg()

In [9]:
def augmentation(input_img):
    list_data=[]
    list_data.append(input_img)


    #Crop
    for i in range(0 , len(list_data)):

        crop2 = iaa.Crop(percent=(0, 0.2)) 
        input_crop2 = crop2.augment_image(list_data[i])
        list_data.append(input_crop2)

    #shearing (random rotate)
    for i in range(0 , len(list_data)):
        rot3 = iaa.Affine(rotate=(-40,20))
        input_rot3 = rot3.augment_image(list_data[i])
        list_data.append(input_rot3)

        rot4 = iaa.Affine(rotate=(40,20))
        input_rot4 = rot4.augment_image(list_data[i])
        list_data.append(input_rot4)

    #noise
    for i in range(0 , len(list_data)):
        noise1=iaa.AdditiveGaussianNoise(10,15)
        input_noise1=noise1.augment_image(list_data[i])
        list_data.append(input_noise1)

        noise2=iaa.AdditiveGaussianNoise(10,8)
        input_noise2=noise2.augment_image(list_data[i])
        list_data.append(input_noise2)

    return list_data

In [10]:
def create_label(image_name):
    img = image_name[0:4]
    if img == 'Bask':
        return 0
    elif img == 'Foot':
        return 1
    elif img == 'Rowi':
        return 2
    elif img == 'Swim':
        return 3
    elif img == 'Tenn':
        return 4
    elif img == 'Yoga':
        return 5

In [11]:
def create_train_data():
    training_data = []
    t = 0
    for img in tqdm(os.listdir(TRAIN_DIR)):
        if (t < 200):
          t+=1
          continue
        path = os.path.join(TRAIN_DIR, img)
        img_data = cv2.imread(path,1)
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)

        label_list = create_label(img)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img_data), label_list])

        #append augmentation to train data
        aug_list = augmentation(img_data);
        for aug in aug_list:
            training_data.append([np.array(aug), label_list])

    print("Train size = " , len(training_data))
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [12]:
column1=[]
def create_test_data():
    test_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
      
        path = os.path.join(TEST_DIR, img)
        img_data = cv2.imread(path,1)
        column1.append(img)
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)

        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        test_data.append([np.array(img_data), 0])

    np.save('test_data.npy', test_data)
    return test_data


In [13]:
def create_validation_data():
    validation_data = []
    t = 0
    for img in tqdm(os.listdir(TRAIN_DIR)):
        if (t > 200):
          break
        t+=1
        path = os.path.join(TRAIN_DIR, img)
        img_data = cv2.imread(path,1)
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)

        label_list = create_label(img)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        validation_data.append([np.array(img_data), label_list])

    shuffle(validation_data)
    np.save('validation_data.npy', validation_data)
    return validation_data

In [14]:
if (os.path.exists('train_data.npy')):
    all_images = np.load('train_data.npy',allow_pickle=True)
else: 
    all_images = create_train_data()

print("all images = ",len(all_images))
for i in range(0 , len(all_images)):
  train_images.append(all_images[i]);

X_train = np.array([i[0] for i in train_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y_train = np.array([i[1] for i in train_images]).reshape(-1,)

#==============================================================================================

if (os.path.exists('validation_data.npy')):
    all_images_val = np.load('validation_data.npy',allow_pickle=True)
else: 
    all_images_val = create_validation_data()
print("all images = ",len(all_images_val))
for i in range(0 , len(all_images_val)):
  val_images.append(all_images_val[i]);

X_val = np.array([i[0] for i in val_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y_val = np.array([i[1] for i in val_images]).reshape(-1,)


#==============================================================================================

if (os.path.exists('test_data.npy')):
    all_images_test = np.load('test_data.npy',allow_pickle=True)
else: 
    all_images_test = create_test_data()
print("all images = ",len(all_images_test))
for i in range(0 , len(all_images_test)):
  test_images.append(all_images_test[i]);

X_test = np.array([i[0] for i in test_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y_test = np.array([i[1] for i in test_images]).reshape(-1,)


all images =  28139
all images =  201
all images =  688


In [21]:

def CNN():
    model1 = Sequential([
            layers.Conv2D(filters = 64 ,kernel_size = (3, 3), input_shape = (IMG_SIZE, IMG_SIZE, 3), padding='same', activation='relu'),
            layers.Conv2D(filters = 64 ,kernel_size = (3, 3), padding='same', activation='relu'),
            layers.MaxPooling2D(pool_size = (2,2)),

            layers.Conv2D(filters = 128 ,kernel_size = (3, 3), padding='same', activation='relu'),
            layers.Conv2D(filters = 128 ,kernel_size = (3, 3), padding='same', activation='relu'),
            layers.MaxPooling2D(pool_size=(2,2)),

            layers.Conv2D(filters = 256 ,kernel_size = (3, 3), padding='same', activation='relu'),
            layers.Conv2D(filters = 256 ,kernel_size = (3, 3), padding='same', activation='relu'),
            layers.Conv2D(filters = 256 ,kernel_size = (3, 3), padding='same', activation='relu'),
            layers.MaxPooling2D(pool_size=(2,2)),

            layers.Conv2D(filters = 256 ,kernel_size = (3, 3), padding='same', activation='relu'),
            layers.Conv2D(filters = 256 ,kernel_size = (3, 3), padding='same', activation='relu'),
            layers.Conv2D(filters = 256 ,kernel_size = (3, 3), padding='same', activation='relu'),
            layers.MaxPooling2D(pool_size=(2,2)),

            layers.Conv2D(filters = 512 ,kernel_size = (3, 3), padding='same', activation='relu'),
            layers.Conv2D(filters = 512 ,kernel_size = (3, 3), padding='same', activation='relu'),
            layers.Conv2D(filters = 512 ,kernel_size = (3, 3), padding='same', activation='relu'),
            layers.MaxPooling2D(pool_size=(2,2)),

            layers.Flatten(),
            layers.Dense(4096, activation='relu'),
            layers.Dense(4096, activation='relu'),
            layers.Dense(1000, activation='relu'),
            #Dropout(0.5),
            layers.Dense(6, activation = 'softmax')
    ])

    model1.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate = 0.00001),
        loss=SparseCategoricalCrossentropy(),
        metrics=['accuracy'])
    
    model1.fit(X_train, y_train, validation_data = (X_val, Y_val), epochs = 5)
    model1.save('drive/MyDrive/model.h5')


if (os.path.exists('drive/MyDrive/model.h5')):
      model1 = load_model('drive/MyDrive/model.h5')
else : CNN()
    


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [23]:
def test():
  Y_test=model1.predict(X_test)
  print(len(X_test))
  prediction=[]
  for i in Y_test:
    prediction.append(np.argmax(i))
  data={'image_name':column1,'label':prediction}
  print(len(column1),len(prediction))
  df=pd.DataFrame(data)
  df.to_csv('drive/MyDrive/output.csv',index=False)
  print(df)

test()

100%|██████████| 688/688 [00:07<00:00, 90.07it/s] 
/usr/local/lib/python3.8/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


688
688 688
    image_name  label
0        0.jpg      4
1        1.jpg      4
2      102.jpg      1
3      100.jpg      1
4       10.jpg      5
..         ...    ...
683     94.jpg      1
684     97.jpg      1
685     96.jpg      4
686     98.jpg      0
687     99.jpg      0

[688 rows x 2 columns]
